In [ ]:
"""This notebook precalculates the masks for ACS shapefiles for 5km data AGCD_05i.
Author: Gen Tolhurst, Email: gentolhurst@gmail.com"""

In [1]:
cd /g/data/mn51/users/gt3409/acs_shapefiles/

/g/data/mn51/users/gt3409/acs_shapefiles


In [4]:
import xarray as xr
import geopandas as gpd
import regionmask
from glob import glob
import numpy as np
import pandas as pd

In [5]:
# Get regionmask object from the shapefiles 

shapefile_list = ["aus_local_gov",
                  "aus_states_territories",
                  "australia", 
                  "broadacre_regions", 
                  "ncra_regions", 
                  "NCRA_regions_coastal_waters_GDA94", 
                  "nrm_regions", ]

name_dict = {"aus_local_gov":"LGA_NAME22", 
             "aus_states_territories":"STE_NAME21",
             "australia": "AUS_NAME21",
             "broadacre_regions": "name",
             "ncra_regions": "regionname", 
             "NCRA_regions_coastal_waters_GDA94": "regionname",
             "nrm_regions":"SubClusNm",
            }


abbr_dict = {"aus_local_gov":"LGA_CODE22", 
             "aus_states_territories":"ABBREV",
             "australia": "AUS_CODE21",
             "broadacre_regions": "aagis",
             "ncra_regions": "short_name", 
             "NCRA_regions_coastal_waters_GDA94": "NCRA",
             "nrm_regions": "SubClusAb",
            }

def get_regions(shapefiles):
    """
    This function takes a list of names of shape files from ia39 and
    returns a combined regionmask.
    
    Parameters
    -----------
    name: str
        one of "aus_local_gov", "aus_states_territories", "australia", 
        "nrm_regions", "ncra_regions","broadacre_regions",
        "NCRA_regions_coastal_waters_GDA94"

    Returns
    -------
    geopandas dataframe
    
    """
    gdfs = {}
    PATH = "/g/data/ia39/aus-ref-clim-data-nci/shapefiles/data"
    
    for i, shapefile in enumerate(shapefiles):
        gdfs[i] = gpd.read_file(glob(f"{PATH}/{shapefile}/*.shp")[0]).rename(columns = {name_dict[shapefile]:"NAME", abbr_dict[shapefile]:"abbrevs"}).to_crs(crs = "GDA2020")
    gdf = pd.concat(gdfs)
    gdf.index = np.arange(0, len(gdf))
    return regionmask.from_geopandas(gdf, names="NAME", abbrevs="abbrevs", name= "-".join(shapefiles), overlap=True) 

In [6]:
# open any dataset to get the lat lon grid for AGCD-05i
ds_agcd05i = xr.open_dataset('/g/data/ia39/australian-climate-service/test-data/CORDEX-CMIP6/bias-adjustment-output/AGCD-05i/BOM/ACCESS-CM2/historical/r4i1p1f1/BARPA-R/v1-r1-ACS-QME-AGCD-1960-2022/day/tasmaxAdjust/tasmaxAdjust_AGCD-05i_ACCESS-CM2_historical_r4i1p1f1_BOM_BARPA-R_v1-r1-ACS-QME-AGCD-1960-2022_day_20140101-20141231.nc')

In [22]:
# Make a netcdf fractional mask file for all shapefiles in list and save it
# This one is especially memory intensive, particularly for the LGA file with many areas
directory = "/g/data/ia39/aus-ref-clim-data-nci/shapefiles/masks/AGCD-05i"

for shp in shapefile_list:
    regions = get_regions([shp])
    mask = regions.mask_3D_frac_approx(ds_agcd05i).to_netcdf(f"{directory}/mask-3D-frac-approx_{shp.replace('_', '-')}.nc")


In [23]:
# Make a netcdf centred mask file for all shapefiles in list and save it
directory = "/g/data/ia39/aus-ref-clim-data-nci/shapefiles/masks/AGCD-05i"

for shp in shapefile_list:
    regions = get_regions([shp])
    mask = regions.mask_3D(ds_agcd05i).to_netcdf(f"{directory}/mask-3D_{shp.replace('_', '-')}.nc")